<a href="https://colab.research.google.com/github/AndrewKostin/TUII_2019/blob/master/umap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

from umap import UMAP
from sklearn.preprocessing import StandardScaler

from google.colab import files, drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path = '/content/drive/My Drive/colab/ONTI_FINAL/{}'

train = pd.read_csv(path.format('transactions_train.csv'))
test = pd.read_csv(path.format('transactions_test.csv'))

target = pd.read_csv(
    path.format('train_target.csv'),
    index_col='client_dk')
target.columns = target.columns.astype(int)

transactions = pd.concat([train, test]).reset_index(drop=True)

In [0]:
big = transactions.groupby('client_dk').amount.agg(['sum', 'count', 'mean', 'min', 'max', 'std']).fillna(0)

In [0]:
v_c = transactions.small_group.value_counts(normalize=True)*100
del_group = v_c[v_c < 0.001].index
transactions = transactions[~transactions.small_group.isin(del_group)]

In [5]:
%%time
# data = transactions.groupby(['client_dk', 'small_group', transactions.trans_date//30]).amount.count().unstack().unstack().fillna(0)
data = transactions.groupby(['client_dk', 'small_group']).amount.count().unstack().fillna(0)

scaler = StandardScaler()
umap = UMAP(n_neighbors=5, n_components=2, min_dist=0.2)#, metric='cosine')
data_umap = umap.fit_transform(scaler.fit_transform(data))

big = big.join(pd.DataFrame(data_umap))

/usr/local/lib/python3.6/dist-packages/numba/typed_passes.py:293: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../usr/local/lib/python3.6/dist-packages/umap/rp_tree.py", line 135:
@numba.njit(fastmath=True, nogil=True, parallel=True)
def euclidean_random_projection_split(data, indices, rng_state):
^

  state.func_ir.loc))
/usr/local/lib/python3.6/dist-packages/umap/nndescent.py:92: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../usr/local/lib/python3.6/dist-packages/umap/utils.py", line 409:
@numba.njit(parallel=True

CPU times: user 1min 45s, sys: 1.42 s, total: 1min 46s
Wall time: 1min 37s


In [0]:
transactions = transactions[transactions.small_group.isin(target.columns)]

In [13]:
transactions.small_group.value_counts()

45    358404
41    344523
27    231076
32    178963
73     68651
67     39381
88     36163
81      7410
Name: small_group, dtype: int64

In [0]:
transactions = transactions.sort_values(by=['client_dk', 'trans_date'])

add = []
for group in transactions.small_group.unique():
    mask = transactions.small_group == group
    t = transactions.loc[mask, ['client_dk', 'trans_date']]
    time_interval = t.trans_date - t.trans_date.shift()
    
    # эти 2 строки заполняют первые значения нанами. 
    # это нужно потому-что в этих значениях интервал между первой покупкой нового глиента и последней покупкой прошлого клиента
    new_client = (t.client_dk - t.client_dk.shift()) > 0
    time_interval.mask(new_client, inplace=True)

    time_interval = time_interval.groupby(t.client_dk).agg(['min', 'max', 'std', 'mean'])
    time_interval.columns += f'_time_delta_{group}'
    add.append(time_interval)

big = big.join(pd.concat(add, axis=1).fillna(-1))    

In [0]:
# add = transactions.groupby(['client_dk', 'small_group', transactions.trans_date//30]).amount.agg(['sum', 'count', 'mean', 'min', 'max', 'std']).unstack().fillna(0)
# add.columns = [f'month_{col1}_{col2}' for col1, col2 in add.columns]

In [9]:
%%time
# данные по цене покупок в каждой группе
add = transactions.groupby(['client_dk', 'small_group']).amount.agg(['sum', 'count', 'mean', 'min', 'max', 'std']).unstack().fillna(0)
add.columns = [f'{col1}_{col2}' for col1, col2 in add.columns]
big = big.join(add)

# тоже самое по датам
add = transactions.groupby(['client_dk', 'small_group']).trans_date.agg(['mean', 'min', 'max', 'std']).unstack().fillna(0)
add.columns = [f'date_{col1}_{col2}' for col1, col2 in add.columns]
big = big.join(add)

# цена покупок по месяцам 
add = transactions.groupby(['client_dk', transactions.trans_date//30]).amount.agg(['sum', 'count', 'mean', 'min', 'max', 'std']).unstack().fillna(0)
add.columns = [f'month_{col1}_{col2}' for col1, col2 in add.columns]
big = big.join(add)

CPU times: user 2.21 s, sys: 63.9 ms, total: 2.27 s
Wall time: 2.27 s


In [10]:
big

,sum,count,mean,min,max,std,0,1,min_time_delta_32,max_time_delta_32,std_time_delta_32,mean_time_delta_32,min_time_delta_45,max_time_delta_45,std_time_delta_45,mean_time_delta_45,min_time_delta_73,max_time_delta_73,std_time_delta_73,mean_time_delta_73,min_time_delta_27,max_time_delta_27,std_time_delta_27,mean_time_delta_27,min_time_delta_41,max_time_delta_41,std_time_delta_41,mean_time_delta_41,min_time_delta_88,max_time_delta_88,std_time_delta_88,mean_time_delta_88,min_time_delta_67,max_time_delta_67,std_time_delta_67,mean_time_delta_67,min_time_delta_81,max_time_delta_81,std_time_delta_81,mean_time_delta_81,...,month_mean_12,month_min_0,month_min_1,month_min_2,month_min_3,month_min_4,month_min_5,month_min_6,month_min_7,month_min_8,month_min_9,month_min_10,month_min_11,month_min_12,month_max_0,month_max_1,month_max_2,month_max_3,month_max_4,month_max_5,month_max_6,month_max_7,month_max_8,month_max_9,month_max_10,month_max_11,month_max_12,month_std_0,month_std_1,month_std_2,month_std_3,month_std_4,month_std_5,month_std_6,month_std_7,month_std_8,month_std_9,month_std_10,month_std_11,month_std_12
client_dk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,21590.476,774,27.894672,0.048,7364.548,269.495266,5.556036,-0.436245,2.0,309.0,121.891345,60.500000,8.0,114.0,45.219465,68.400000,8.0,306.0,210.717821,157.000000,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.0,-1.00,-1.0,-1.0,-1.00000,-1.00,-1.0,-1.0,-1.0,-1.0,...,13.439333,21.271,21.560,9.668,0.000,0.000,25.666,0.000,0.000,0.000,30.799,35.933,24.171,4.834,30.799,21.560,9.668,0.000,0.000,25.666,0.000,0.000,0.000,35.933,35.933,24.171,35.933,4.884041,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.630286,0.000000,0.000000,11.316217
1,39375.929,1251,31.475563,0.048,1967.723,84.370225,9.699308,-3.007316,44.0,109.0,28.324018,70.750000,7.0,136.0,52.624772,58.833333,-1.0,-1.0,-1.000000,-1.000000,113.0,142.0,20.506097,127.500000,0.0,82.0,29.592470,23.000000,-1.0,-1.0,-1.0,-1.00,-1.0,-1.0,-1.00000,-1.00,-1.0,-1.0,-1.0,-1.0,...,28.296667,40.039,82.132,48.342,0.000,39.013,507.592,123.711,24.171,44.475,14.503,48.342,5.544,27.072,55.110,513.323,58.519,0.000,96.684,507.592,123.711,24.171,44.475,1145.708,48.342,76.999,28.909,7.548517,304.898080,7.196226,0.000000,40.779555,0.000000,0.000000,0.000000,0.000000,799.882726,0.000000,27.804335,1.060592
2,40328.909,1514,26.637324,0.048,1280.434,56.031283,0.479952,1.820898,0.0,0.0,-1.000000,0.000000,7.0,35.0,10.768119,17.571429,-1.0,-1.0,-1.000000,-1.000000,14.0,61.0,18.058547,32.888889,0.0,114.0,34.427149,16.857143,-1.0,-1.0,-1.0,-1.00,-1.0,-1.0,-1.00000,-1.00,-1.0,-1.0,-1.0,-1.0,...,48.485444,0.000,16.920,24.171,15.400,50.759,20.533,29.005,72.513,12.320,20.533,0.000,8.213,9.185,0.000,68.066,24.171,35.933,50.759,30.799,29.005,72.513,29.005,20.533,0.000,61.599,169.197,0.000000,23.446011,0.000000,7.968146,0.000000,5.927078,0.000000,0.000000,6.982558,0.000000,0.000000,14.030348,62.715230
3,49014.558,925,52.988711,0.048,892.156,105.198624,3.434414,0.741731,27.0,311.0,200.818326,169.000000,6.0,85.0,28.636912,29.454545,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000000,-1.0,-1.0,-1.000000,-1.000000,0.0,0.0,-1.0,0.00,-1.0,-1.0,-1.00000,-1.00,-1.0,-1.0,-1.0,-1.0,...,66.732333,0.000,313.127,6.160,230.996,19.820,24.640,36.740,0.000,0.000,29.259,20.787,53.176,21.560,0.000,313.127,266.928,230.996,19.820,41.579,41.574,0.000,0.000,29.259,20.787,53.176,153.997,0.000000,0.000000,124.724428,0.000000,0.000000,8.532774,3.418154,0.000000,0.000000,0.000000,0.000000,0.000000,75.589107
4,52078.639,1412,36.882889,0.048,1365.389,74.227945,2.698409,-4.700049,-1.0,-1.0,-1.000000,-1.000000,0.0,53.0,20.413231,16.500000,-1.0,-1.0,-1.000000,-1.000000,71.0,71.0,-1.000000,71.000000,0.0,84.0,9.905750,3.226087,0.0,0.0,-1.0,0.00,-1.0,-1.0,-1.00000,-1.00,-1.0,-1.0,-1.0,-1.0,...,27.367909,7.251,7.700,24.171,29.005,12.086,12.833,3.867,4.834,31.422,8.702,7.251,29.972,5.133,100.098,30.799,24.171,29

In [0]:
big.to_csv(path.format('big.csv'))